In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os
%matplotlib inline

In [81]:
cwd = os.getcwd()
data_dir = os.path.join(os.pardir, 'data', 'whale')
img_dir = os.path.join(os.pardir, 'data', 'whale', 'imgs')


In [8]:
data_dir

'../data/whale'

In [9]:
tr = pd.read_csv(data_dir+'/train.csv')

In [11]:
sub = pd.read_csv(data_dir+ '/sample_submission.csv')

In [12]:
sub.shape

(6925, 448)

In [13]:
tr.whaleID.nunique()

447

In [14]:
len(tr)

4544

### Make the test directory and move the test files to that directory.

In [16]:
% cd $data_dir
% mv w_7489.jpg imgs/

/home/ubuntu/courses/deeplearning1/nbs/data/whale


In [20]:
% cd imgs

/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs


In [ ]:
% mkdir vgg
% mkdir vgg/test
% mkdir vgg/train
% mkdir vgg/valid

In [23]:
os.getcwd()

'/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs'

In [33]:
imgs = glob('*.jpg')

In [25]:
len(imgs)

11469

In [34]:
files = pd.DataFrame(imgs, columns=['filename'])

In [35]:
files.head()

,filename
0,w_8037.jpg
1,w_2255.jpg
2,w_4494.jpg
3,w_3674.jpg
4,w_4673.jpg


In [28]:
test = files[~files.filename.isin(tr.Image)]

In [36]:
test.shape

(6925, 1)

In [37]:
os.path.join(os.getcwd(), test.iloc[0].filename)

'/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs/w_8037.jpg'

In [39]:
os.path.join(os.getcwd(), 'vgg', 'test', test.iloc[0].filename)

'/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs/vgg/test/w_8037.jpg'

In [40]:
for fname in test.filename:
    os.rename(os.path.join(os.getcwd(), fname),os.path.join(os.getcwd(), 'vgg', 'test', fname))


In [41]:
% cd vgg/test

/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs/vgg/test


In [42]:
% mkdir unknown

In [43]:
% mv *.jpg unknown/

In [44]:
% cd unknown

/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs/vgg/test/unknown


In [45]:
test_files = glob('*.jpg')

In [46]:
assert len(test_files) == len(sub)

### Make train and validation sets

In [30]:
tr.shape

(4544, 2)

In [31]:
tr.head()

,Image,whaleID
0,w_7812.jpg,whale_48813
1,w_4598.jpg,whale_09913
2,w_3828.jpg,whale_45062
3,w_8734.jpg,whale_74162
4,w_3251.jpg,whale_99558


In [47]:
for_train = []
tr = tr.sample(frac=1)
for w in tr.groupby('whaleID'):
    for_train.append(w[1].iloc[0,:].Image)

In [48]:
len(for_train)

447

In [49]:
left = tr[~tr.Image.isin(for_train)]

In [50]:
val = left.sample(n=900)

In [51]:
val.whaleID.nunique()

345

In [52]:
train_imgs = left[~left.Image.isin(val.Image)]

In [53]:
len(train_imgs) + len(for_train) + len(val) == len(tr)

True

In [54]:
train = np.concatenate([for_train, train_imgs.Image.values])

In [55]:
len(train)

3644

In [56]:
len(val)

900

### Create directories for the whales in the train and validation directories and move the images to their correct whale directories

In [57]:
% pwd

u'/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs/vgg/test/unknown'

In [58]:
% cd ../../../../
% pwd

/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs/vgg


u'/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs/vgg'

In [ ]:
% mv *.jpg vgg/train/

In [62]:
tr_df = tr[tr.Image.isin(train)]

In [63]:
tr_df.shape

(3644, 2)

In [64]:
% cd vgg/train

/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs/vgg/train


In [65]:
for whale in tr_df.whaleID.unique():
    os.mkdir(whale)

In [67]:
for r in tr_df.iterrows():
    folder = r[1].whaleID
    fname = r[1].Image
    try:
        os.rename(os.path.join(os.getcwd(), fname), os.path.join(os.getcwd(), folder, fname))
    except:
        print(r)

In [68]:
% pwd

u'/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs/vgg/train'

In [69]:
% cd ../valid

/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs/vgg/valid


In [70]:
for whale in val.whaleID.unique():
    os.mkdir(whale)

In [71]:
% cd ../train

/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs/vgg/train


In [73]:
i = 0
for r in val.iterrows():
    folder = r[1].whaleID
    fname = r[1].Image
    #i +=1
    #if i > 3: break
    try:
        #print(os.path.join(os.pardir, 'valid', folder, fname))
        os.rename(os.path.join(os.getcwd(), fname), os.path.join(os.pardir, 'valid', folder, fname))
    except:
        print(r)

In [74]:
% pwd

u'/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs/vgg/train'

In [75]:
sum([len(files) for r, d, files in os.walk(os.getcwd())])

3644

In [76]:
sum([len(files) for r, d, files in os.walk(os.path.join(os.pardir, 'valid'))])

900

In [77]:
sum([len(files) for r, d, files in os.walk(os.path.join(os.pardir, 'test', 'unknown'))])

6925

# Make directories for whales not including in our validation set so that we can fit models

In [84]:
% cd ..

/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs/vgg


In [85]:
train_dirs = os.listdir(os.getcwd()+'/train')
valid_dirs = os.listdir(os.getcwd()+'/valid')

In [90]:
len(train_dirs)
len(valid_dirs)

345

In [88]:
% cd valid

/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs/vgg/valid


In [92]:
i = 0
for dr in train_dirs:
    if dr not in valid_dirs:
        dest = os.getcwd() +'/'+ dr
        #print(dest)
        #i+=1
        #if i > 3: break
        os.mkdir(dest)
